In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '..')

from lib import models, graph, coarsening, utils
import tensorflow as tf
import numpy as np
import time
import scipy.sparse
from scipy.io import *
%matplotlib inline


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Graphs.
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
# TODO: change cgcnn for combinatorial Laplacians.
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
flags.DEFINE_integer('coarsening_levels', 4, 'Number of coarsened graphs.')

# Directories.
flags.DEFINE_string('dir_data', os.path.join('..', 'data', 'mnist'), 'Directory to store data.')

# Load Data

In [3]:
# Number of Nearest Neighbors
k = 110

base_storage_dir = '/data4/abhijeet/Datasets/PASCAL_VOC/GCN/preprocessing/experiment3/'
temp_filename = base_storage_dir + 'scikit_fc7_k_' + str(k) + '.mat'

if os.path.exists(temp_filename):
    print('File already exists\n No Preprocessing Required')
else:
    print('Do preprocessing --> TODO --> call the fucntion with a specific parameter')
    
data = scipy.io.loadmat( temp_filename)

X_train          = data['X_train']
X_test           = data['X_test']
train_labels     = data['Y_train']
test_labels      = data['Y_test']
Adjacency_test   = data['Adjacency_test']
Adjacency_train  = data['Adjacency_train']

#delete unused variables
del data

File already exists
 No Preprocessing Required


In [4]:
#define laplacians, coarsen adjacency, permutate data points.
train_laplacians = [[] for i in range(1 + FLAGS.coarsening_levels)]
test_laplacians = [[] for i in range(1 + FLAGS.coarsening_levels)]

train_perm ,test_perm = [] ,[]

for i in range(X_train.shape[0]):
    if not(i%1000):
        print(i)
    graphs, temp_perm = coarsening.coarsen(Adjacency_train[0][i], levels=FLAGS.coarsening_levels, self_connections=False)
    train_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(1 + FLAGS.coarsening_levels):
        train_laplacians[j].append(temp_L[j])     

for i in range(X_test.shape[0]):
    if not(i%1000):
        print(i)
    graphs, temp_perm = coarsening.coarsen(Adjacency_test[0][i], levels=FLAGS.coarsening_levels, self_connections=False)
    test_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(1 + FLAGS.coarsening_levels):
        test_laplacians[j].append(temp_L[j])     

#del Adjacency_test, Adjacency_train

0
1000
2000
3000
4000
5000
0
1000
2000
3000
4000


In [5]:
print(type(test_laplacians))
print(len(test_laplacians))
for i in range(4):
    print(len(test_laplacians[i]))
    print(test_laplacians[i][0].shape)

<class 'list'>
5
4952
(112, 112)
4952
(56, 56)
4952
(28, 28)
4952
(14, 14)


In [6]:
print(len(train_perm[0]),type(train_perm))
train_data = np.zeros((X_train.shape[0], len(train_perm[0]), X_train.shape[2]))
test_data = np.zeros((X_test.shape[0], len(test_perm[0]), X_test.shape[2]))
for i in range(train_data.shape[0]):
    if not(i%1000):
        print(i)
    train_data[i,:,:] = coarsening.perm_data_point(X_train[i,:,:], train_perm[i])

for i in range(test_data.shape[0]):
    if not(i%1000):
        print(i)
    test_data[i,:,:] = coarsening.perm_data_point(X_test[i,:,:], test_perm[i])

val_data = test_data.copy()
val_laplacians = test_laplacians.copy()
val_labels = test_labels.copy()

#del X_train, Y_train, X_test, Y_test 
#del perm_train, perm_test

112 <class 'list'>
0
1000
2000
3000
4000
5000
0
1000
2000
3000
4000


In [7]:
print(len(test_laplacians))
#print(test_laplacians[0])
L = []
for i in range(FLAGS.coarsening_levels):
    L.append(train_laplacians[i][0])
L = np.array(L)
print(L)
for i in range(4):
    print(L[i].shape)

5
[ array([[-0.00916256, -0.00910209, -0.00910398, ..., -0.00909108,
        -0.00908656, -0.00909522],
       [-0.00910209, -0.00916452, -0.00909317, ..., -0.00908922,
        -0.00908687, -0.00909534],
       [-0.00910398, -0.00909317, -0.00916536, ..., -0.00909255,
        -0.00908611, -0.0090926 ],
       ..., 
       [-0.00909108, -0.00908922, -0.00909255, ..., -0.00915947,
        -0.00909905, -0.00909837],
       [-0.00908656, -0.00908687, -0.00908611, ..., -0.00909905,
        -0.00916364, -0.00908909],
       [-0.00909522, -0.00909534, -0.0090926 , ..., -0.00909837,
        -0.00908909, -0.00916216]])
 array([[-0.01826563, -0.01819792, -0.01819491, ..., -0.01817355,
        -0.0181781 , -0.018182  ],
       [-0.01819792, -0.01826408, -0.01818837, ..., -0.01816765,
        -0.01817973, -0.01817587],
       [-0.01819491, -0.01818837, -0.01826495, ..., -0.01817989,
        -0.01818951, -0.01818773],
       ..., 
       [-0.01817355, -0.01816765, -0.01817989, ..., -0.01825477,
   

# GCN 

In [19]:
from lib import models, graph, coarsening, utils
common = {}
common['dir_name']       = 'cifar10/'
common['num_epochs']     = 20
common['batch_size']     = 100
common['decay_steps']    = (train_data.shape[0] + val_data.shape[0]) / common['batch_size']
common['eval_frequency'] = 10 * common['num_epochs']
common['brelu']          = 'b1relu'
common['pool']           = 'mpool1'
num_labels_per_image     = 2
C = train_labels.shape[1]  # number of classes
model_perf = utils.model_perf()

In [9]:
if True:
    name = 'softmax'
    params = common.copy()
    params['dir_name'] += name
    params['regularization'] = 5e-4
    params['dropout']        = 1
    params['learning_rate']  = 0.02
    params['decay_rate']     = 0.95
    params['momentum']       = 0.9
    params['F']              = []
    params['F_0']            = 20
    params['K']              = []
    params['p']              = []
    params['M']              = [C] 
    params['train_laplacians'] = train_laplacians
    params['test_laplacians'] = val_laplacians
    params['val_laplacians'] = test_laplacians
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: logits (softmax)
    representation: M_1 = 20
    weights: M_0 * M_1 = 112 * 20 = 2240
    biases: M_1 = 20
step 2000 / 10022 (epoch 39.91 / 200):
  learning_rate = 7.17e-03, loss_average = 1.00e+00
5595.0 / 7013 = 0.797804078141
  validation precision: 0.57, recall : 0.75, f_measure: 0.65, mAP 0.93, MAP 0.88
  time: 30s (wall 31s)
step 4000 / 10022 (epoch 79.82 / 200):
  learning_rate = 2.57e-03, loss_average = 8.98e-01
5573.0 / 7013 = 0.794667046913
  validation precision: 0.57, recall : 0.75, f_measure: 0.64, mAP 0.93, MAP 0.88
  time: 44s (wall 40s)
step 6000 / 10022 (epoch 119.74 / 200):
  learning_rate = 9.21e-04, loss_average = 8.60e-01
5579.0 / 7013 = 0.795522600884
  validation precision: 0.57, recall : 0.75, f_measure: 0.64, mAP 0.93, MAP 0.88
  time: 58s (wall 49s)
step 8000 / 10022 (epoch 159.65 / 200):
  learning_rate = 3.30e-04, loss_average = 8.51e-01
5578.0 / 7013 = 0.795380008556
  validation precision: 0.57, recall : 0.75,

In [10]:
print(np.sum(train_labels),np.sum(test_labels))

7584 7013


In [11]:
# Common hyper-parameters for networks with one convolutional layer.
common['regularization'] = 0
common['dropout']        = 1
common['learning_rate']  = 0.02
common['decay_rate']     = 0.95
common['momentum']       = 0.9
common['F']              = [10]
common['F_0']            = 20
common['K']              = [100]
common['p']              = [4]
common['M']              = [C]
common['train_laplacians'] = train_laplacians
common['test_laplacians'] = val_laplacians
common['val_laplacians'] = test_laplacians

In [12]:
# With 'chebyshev2' and 'b2relu', it corresponds to cgcnn2_2(L[0], F=10, K=20).
if True:
    name = 'cgconv_softmax'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
#    params['filter'] = 'chebyshev2'
#    params['brelu'] = 'b2relu'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 112 * 10 / 4 = 280
    weights: F_0 * F_1 * K_1 = 20 * 10 * 100 = 20000
    biases: F_1 = 10
  layer 2: logits (softmax)
    representation: M_2 = 20
    weights: M_1 * M_2 = 280 * 20 = 5600
    biases: M_2 = 20
0 th conv layer 100 112 112
step 2000 / 10022 (epoch 39.91 / 200):
  learning_rate = 7.17e-03, loss_average = 1.74e+00
5314.0 / 7013 = 0.757735633823
  validation precision: 0.53, recall : 0.69, f_measure: 0.60, mAP 0.91, MAP 0.85
  time: 114s (wall 220s)
step 4000 / 10022 (epoch 79.82 / 200):
  learning_rate = 2.57e-03, loss_average = 1.51e+00
5299.0 / 7013 = 0.755596748895
  validation precision: 0.52, recall : 0.70, f_measure: 0.60, mAP 0.92, MAP 0.84
  time: 192s (wall 395s)
step 6000 / 10022 (epoch 119.74 / 200):
  learning_rate = 9.21e-04, loss_average = 1.53e+00
5326.0 / 7013 = 0.759446741765
  validation precision: 0.52, recall : 0.70, f_measure: 0.60, mAP 0.92, MAP 0.85
  time: 

In [17]:
# Common hyper-parameters for LeNet5-like networks.
common['regularization'] = 5e-4
common['dropout']        = 0.5
common['learning_rate']  = 0.02  # 0.03 in the paper but sgconv_sgconv_fc_softmax has difficulty to converge
common['decay_rate']     = 0.995
common['momentum']       = 0.9
common['F']              = [20,40,60]#[32, 64]
common['F_0']            = 20
common['K']              = [15,15,15]#[25, 25]
common['p']              = [2, 2, 2] #[2,2]
common['M']              = [C]#[512, C]
common['train_laplacians'] = train_laplacians
common['test_laplacians'] = val_laplacians
common['val_laplacians'] = test_laplacians

In [18]:
if True:
    name = 'cgconv_cgconv_fc_softmax'  # 'Chebyshev'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 112 * 20 / 2 = 1120
    weights: F_0 * F_1 * K_1 = 20 * 20 * 15 = 6000
    biases: F_1 = 20
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 56 * 40 / 2 = 1120
    weights: F_1 * F_2 * K_2 = 20 * 40 * 15 = 12000
    biases: F_2 = 40
  layer 3: cgconv3
    representation: M_2 * F_3 / p_3 = 28 * 60 / 2 = 840
    weights: F_2 * F_3 * K_3 = 40 * 60 * 15 = 36000
    biases: F_3 = 60
  layer 4: logits (softmax)
    representation: M_4 = 20
    weights: M_3 * M_4 = 840 * 20 = 16800
    biases: M_4 = 20
0 th conv layer 100 112 112
1 th conv layer 100 56 56
2 th conv layer 100 28 28
step 2000 / 10022 (epoch 39.91 / 200):
  learning_rate = 1.91e-28, loss_average = 2.39e+00
5107.0 / 7013 = 0.728219021817
  validation precision: 0.50, recall : 0.62, f_measure: 0.55, mAP 0.80, MAP 0.82
  time: 67s (wall 57s)
step 4000 / 10022 (epoch 79.82 / 200):
  learning_rate = 0.00e+00, loss_average = 2.44e+00
51

In [20]:
# Common hyper-parameters for LeNet5-like networks.
common['regularization'] = 5e-4
common['dropout']        = 0.5
common['learning_rate']  = 0.02  # 0.03 in the paper but sgconv_sgconv_fc_softmax has difficulty to converge
common['decay_rate']     = 0.95
common['momentum']       = 0.9
common['F']              = [20,40,60]#[32, 64]
common['F_0']            = 20
common['K']              = [15,15,15]#[25, 25]
common['p']              = [2, 2, 2] #[2,2]
common['M']              = [C]#[512, C]
common['train_laplacians'] = train_laplacians
common['test_laplacians'] = val_laplacians
common['val_laplacians'] = test_laplacians
if True:
    name = 'cgconv_cgconv_fc_softmax'  # 'Chebyshev'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 112 * 20 / 2 = 1120
    weights: F_0 * F_1 * K_1 = 20 * 20 * 15 = 6000
    biases: F_1 = 20
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 56 * 40 / 2 = 1120
    weights: F_1 * F_2 * K_2 = 20 * 40 * 15 = 12000
    biases: F_2 = 40
  layer 3: cgconv3
    representation: M_2 * F_3 / p_3 = 28 * 60 / 2 = 840
    weights: F_2 * F_3 * K_3 = 40 * 60 * 15 = 36000
    biases: F_3 = 60
  layer 4: logits (softmax)
    representation: M_4 = 20
    weights: M_3 * M_4 = 840 * 20 = 16800
    biases: M_4 = 20
0 th conv layer 100 112 112
1 th conv layer 100 56 56
2 th conv layer 100 28 28
step 200 / 1002 (epoch 3.99 / 20):
  learning_rate = 1.00e-03, loss_average = 2.58e+00
4934.0 / 7013 = 0.70355054898
  validation precision: 0.41, recall : 0.55, f_measure: 0.47, mAP 0.67, MAP 0.78
  time: 11s (wall 9s)
step 400 / 1002 (epoch 7.98 / 20):
  learning_rate = 1.25e-07, loss_average = 2.62e+00
4947.0 / 701